# CIFAR–10 Codealong

Codealong of [Radek Osmulski's notebook](https://github.com/radekosmulski/cifar10_docker/blob/master/workspace/cifar10_fastai_dawnbench.ipynb) establishing a CIFAR-10 baseline with the Fastai ImageNet WideResNet22. For a [Fastai CV research collaboration](http://forums.fast.ai/t/research-collaboration-opportunity-with-leslie-smith/16454/76).

Wayne Nixalo –– 2018/6/1

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
# fastai/imagenet-fast/cifar10/models/ repo
from imagenet_fast_cifar_models.wideresnet import wrn_22
from torchvision import transforms, datasets
# allows you to enable the inbuilt cudnn auto-tuner to find the 
# best algorithm for your hardware. https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936/2
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10")

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

We construct the data object manually from low level components in a way that can be used with the fastsai library.

In [4]:
## Aside: checking the normalization transforms
tensor = T(np.ones((3,32,32)))
t1 = transforms.Normalize(stats[0], stats[1])(tensor)
t2 = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))(tensor)
np.unique(np.isclose(t1, t2))

array([ True])

In [4]:
def get_loaders(bs, num_workers):
    traindir = str(PATH/'train')
    valdir   = str(PATH/'test')
    tfms     = [transforms.ToTensor(),
                transforms.Normalize(stats[0], stats[1])]
                #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
    aug_tfms = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
        ] + tfms)
    
    train_dataset = datasets.ImageFolder(traindir, aug_tfms)
    val_dataset   = datasets.ImageFolder(valdir, transforms.Compose(tfms))
    aug_dataset   = datasets.ImageFolder(valdir, aug_tfms)
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader   = torch.utils.data.DataLoader(
        val_dataset, batch_size=bs, shuffle=False, num_workers=num_workers, pin_memory=True)
    aug_loader   = torch.utils.data.DataLoader(
        aug_dataset, batch_size=bs, shuffle=False, num_workers=num_workers, pin_memory=True)
    
    return train_loader, val_loader, aug_loader

This is very similar to how [Fastai initializes its `ModelData`](https://github.com/fastai/fastai/blob/HEAD/fastai/dataset.py#L301-L301), except Fastai uses the Pytorch default `pin_memory=False`.

[What is the disadvantage of using `pin_memory`?](https://discuss.pytorch.org/t/what-is-the-disadvantage-of-using-pin-memory/1702) – 
Pytorch docs:
> by pinning your batch in cpu memory, data transfer to gpu can be much faster

Soumith:
> pinned memory is page-locked memory. It's easy to shoot yourself in the foot if you enable it for everything because it can't be pre-empted. ... if you're seeing system freeze or swap being used a lot, disable it.

In [5]:
def get_data(bs, num_workers):
    trn_dl, val_dl, aug_dl = get_loaders(bs, num_workers)
    data = ModelData(PATH, trn_dl, val_dl)
    data.aug_dl = aug_dl
    data.sz = 32
    return data

def get_learner(arch, bs):
    learn = ConvLearner.from_model_data(arch.cuda(), get_data(bs, num_cpus()))
    learn.crit = nn.CrossEntropyLoss()
    learn.metrics = [accuracy]
    return learn

def get_TTA_accuracy(learn):
    preds, targs = learn.TTA()
    # combining the predictions across augmented and non augmented inputs
    preds = 0.6 * preds[0] + 0.4 * preds[1:].sum(0)
    return accuracy_np(preds, targs)

In [10]:
learner = get_learner(wrn_22(), 512)

In [16]:
x,y = next(iter(learner.data.trn_dl))

In [22]:
type(x[0]), type(y[0])

(torch.FloatTensor, int)

## Fastai DAWN Bench submission

My copy of Radek's reimplementation of the FastAI DAWN Bench submission in terms of archutecture and training parameters – from the [imagenet-fast](https://github.com/fastai/imagenet-fast) repo.

In [6]:
learner = get_learner(wrn_22(), 512)
learner.lr_find(wds=1e-4)
learner.sched.plot(n_skip_end=1)

TypeError: eq received an invalid combination of arguments - got (torch.LongTensor), but expected one of:
 * (int value)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
 * (torch.cuda.LongTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.LongTensor[0m)
